# <center> VICAN: Tutorial</center>



In [3]:
import os
import torch
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from shapely.geometry import Polygon

from vican.cam import estimate_pose_mp
from vican.bipgo import bipartite_se3sync, object_bipartite_se3sync
from vican.plot import plot2D
from vican.geometry import optimize_gauge_SE3, distance_SO3, angle
from vican.dataset import Dataset

# Edit path to the folder containing the renders
DATASET_PATH = "./small_room"
# Edit path to the folder containing the cube calibration images.
OBJ_DATASET_PATH = "./cube_calib"
# Edit marker size in meters
MARKER_SIZE = 0.48 * 0.575
# Check which IDs are used 
MARKER_IDS = list(map(str, range(24)))

dataset     = Dataset(root=DATASET_PATH)
obj_dataset = Dataset(root=OBJ_DATASET_PATH)

# 1. Calibrate object: cube with 24 markers

In [ ]:
# This will compute camera-marker edges via PnP, in parallel
aux = estimate_pose_mp(cams=obj_dataset.im_data['cam'],
                       im_filenames=obj_dataset.im_data['filename'],
                       aruco='DICT_4X4_1000',
                       marker_size=MARKER_SIZE,
                       corner_refine='CORNER_REFINE_APRILTAG',
                       marker_ids=MARKER_IDS,
                       flags='SOLVEPNP_IPPE_SQUARE',
                       brightness=-150,
                       contrast=120)

# Save it to use later, if necessary
torch.save(aux, os.path.join(OBJ_DATASET_PATH, 'cam_marker_edges.pt'))

# Alternatively, comment the code above and load already precomputed edges
#aux = torch.load(os.path.join(OBJ_DATASET_PATH, 'cam_marker_edges.pt'))

# Optimization - see extended paper
obj_pose_est = object_bipartite_se3sync(aux,
                                        noise_model_r=lambda edge : 0.01 * Polygon(zip(edge['corners'][:,0], edge['corners'][:,1])).area**2,
                                        noise_model_t=lambda edge : 0.001 * Polygon(zip(edge['corners'][:,0], edge['corners'][:,1])).area**6,
                                        edge_filter=lambda edge : edge['reprojected_err'] < 0.1,
                                        maxiter=4,
                                        lsqr_solver="conjugate_gradient",
                                        dtype=np.float64)

# 2. Detect markers & estimate camera-marker poses

In [ ]:
# This will compute camera-marker edges via PnP, in parallel
cam_marker_edges = estimate_pose_mp(cams=dataset.im_data['cam'],
                                    im_filenames=dataset.im_data['filename'],
                                    aruco='DICT_4X4_1000',
                                    marker_size=MARKER_SIZE,
                                    corner_refine='CORNER_REFINE_APRILTAG',
                                    marker_ids=MARKER_IDS,
                                    flags='SOLVEPNP_IPPE_SQUARE',
                                    brightness=-150,
                                    contrast=120)

# Save it to use later, if necessary
torch.save(cam_marker_edges, os.path.join(DATASET_PATH, 'cam_marker_edges.pt'))

# Alternatively, comment the code above and load already precomputed edges
# cam_marker_edges = torch.load(os.path.join(RENDER_PATH, 'cam_marker_edges.pt'))

# 3. Optimization

In [ ]:
# Select a subset of timesteps
tmax  = 2000
edges = {k : v for k, v in cam_marker_edges.items() if int(k[1].split('_')[0]) < tmax}

# Optimization - see extended paper
pose_est = bipartite_se3sync(edges,
                             constraints=obj_pose_est,
                             noise_model_r=lambda edge : 0.001 * Polygon(zip(edge['corners'][:,0], edge['corners'][:,1])).area**1.0,
                             noise_model_t=lambda edge : 0.001 * Polygon(zip(edge['corners'][:,0], edge['corners'][:,1])).area**2.0,
                             edge_filter=lambda edge : edge['reprojected_err'] < 0.05,
                             maxiter=4,
                             lsqr_solver="conjugate_gradient",
                             dtype=np.float32)

# 4. Comparison with ground-truth

In [ ]:
missing_cam_ids = [c for c in dataset.cams.keys() if c not in pose_est.keys()]
valid_cam_ids   = [c for c in dataset.cams.keys() if c in pose_est.keys()]

# Compute gauge symmetry in order to compare 
G = optimize_gauge_SE3([dataset.cams[c].extrinsics.inv() for c in valid_cam_ids],
                       [pose_est[c].inv() for c in valid_cam_ids])

r_err, t_err = [], []
x_err, y_err, z_err = [], [], []

for c in valid_cam_ids:
    gt  = dataset.cams[c].extrinsics
    est = G.inv() @ pose_est[c]
    t_err.append(np.linalg.norm(gt.t() - est.t(), ord=2)*100)
    r_err.append(distance_SO3(gt.R(), est.R()))              
    x_err.append(abs(gt.t()[0] - est.t()[0])*100)
    y_err.append(abs(gt.t()[1] - est.t()[1])*100)
    z_err.append(abs(gt.t()[2] - est.t()[2])*100)

print("Missing cameras: {}".format(missing_cam_ids if len(missing_cam_ids) > 0 else "None"))
print("SO(3)\t min: {:.3f}deg | avg: {:.3f}deg | std: {:.3f}cm |  median: {:.3f}deg |  max: {:.3f}deg".format(np.min(r_err), np.mean(r_err), np.std(r_err), np.median(r_err), np.max(r_err)))
print("E(3) \t min: {:.3f}cm  | avg: {:.3f}cm  | std: {:.3f}cm |  median: {:.3f}cm  |  max: {:.3f}cm".format(np.min(t_err), np.mean(t_err), np.std(t_err), np.median(t_err), np.max(t_err)))
print("X \t min: {:.3f}cm  | avg: {:.3f}cm  | std: {:.3f}cm |  median: {:.3f}cm  |  max: {:.3f}cm".format(np.min(x_err), np.mean(x_err), np.std(x_err), np.median(x_err), np.max(x_err)))
print("Y \t min: {:.3f}cm  | avg: {:.3f}cm  | std: {:.3f}cm |  median: {:.3f}cm  |  max: {:.3f}cm".format(np.min(y_err), np.mean(y_err), np.std(y_err), np.median(y_err), np.max(y_err)))
print("Z \t min: {:.3f}cm  | avg: {:.3f}cm  | std: {:.3f}cm |  median: {:.3f}cm  |  max: {:.3f}cm".format(np.min(z_err), np.mean(z_err), np.std(z_err), np.median(z_err), np.max(z_err)))

# 5. 2D Plot

In [ ]:
sns.set_theme()
fig = plt.figure(figsize=(14,14))
ax = fig.add_subplot(111)

plot2D(ax, pose_est, idx=valid_cam_ids, left_gauge=G.inv(), view='xy', marker='x', s=30, c='blue')
plot2D(ax, dataset.cams, view='xy', marker='x', s=30, c='red')
plot2D(ax, dataset.object, view='xy', marker='.', s=15, c=[0,0.6,0,0.4])
plt.axis('equal')
plt.xlabel('x (m)')
plt.ylabel('y (m)')
plt.legend(['Estimates', 'Ground-truth', 'Object'])